# 4장 나이브 베이스: 확률 이론으로 분류하기

In [20]:
import os
srcDir = "C:\Users\BMW\machinelearninginaction-master"
ch4dd = os.path.join(srcDir, 'Ch04')
%cd {ch4dd}

C:\Users\BMW\machinelearninginaction-master\Ch04


## 4.1 베이지안 의사결정 이론으로 분류하기

## 4.2 조건부 확률

## 4.3 조건부 확률로 분류하기

## 4.4 나이브 베이스로 문서 분류하기

## 4.5 파이썬으로 텍스트 분류하기

### 4.5.1 준비: 텍스트로 단어 벡터 만들기

#### 리스팅 4.1 벡터 함수의 단어 목록_p85

In [27]:
import bayes

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] # 1:폭력적인, 0:폭력적이지 않음
    return postingList, classVec
    
# 유일한 단어 집합으로 변환하는 함수
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)


# 문서(vocalList) 내에 어휘 목록(inputSet)의 단어가 존재하는지 여부를 0과 1의 배열로 생성하는 함수
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print "the word: %s is not in my Vocabulary!" % word
        return returnVec

    
# 소스코드 출력    
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
print myVocabList

['cute', 'love', 'help', 'garbage', 'quit', 'I', 'problems', 'is', 'park', 'stop', 'flea', 'dalmation', 'licks', 'food', 'not', 'him', 'buying', 'posting', 'has', 'worthless', 'ate', 'to', 'maybe', 'please', 'dog', 'how', 'stupid', 'so', 'take', 'mr', 'steak', 'my']


In [24]:
# setOfWords2Vec() 확인

print bayes.setOfWords2Vec(myVocabList, listOPosts[0])
print bayes.setOfWords2Vec(myVocabList, listOPosts[1])
print bayes.setOfWords2Vec(myVocabList, listOPosts[2])
print bayes.setOfWords2Vec(myVocabList, listOPosts[3])

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


### 4.5.2 훈련: 단어 벡터로 확률 계산하기

#### 리스팅 4.2 나이브 베이스 분류기 훈련 함수_p89

In [29]:
from numpy import *


#input : 문서 행렬(trainMatrix) => [[문서1], [문서2], ...]
#        문서에 대한 분류 항목(trainCategory) => [0,1,0,1,0,1]
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    
    pAbusive = sum(trainCategory) / float(numTrainDocs) # (폭력적 문서 갯수) / (전체 문서 갯수)
    
    p0Num = zeros(numWords); p1Num = zeros(numWords) # 분자 초기화
    p0Denom = 0.0; p1Denom = 0.0 # 분모 초기화
    
    # 문서 전체 탐색
    for i in range(numTrainDocs):
        if trainCategory[i] == 1: # 그 문서가 폭력적 문서이면.
            p1Num += trainMatrix[i] # p1Num배열에서 해당 문서의 단어를 포함하는 인덱스 +1
            p1Denom += sum(trainMatrix[i]) # 해당 문서 내 단어 총갯수
        else: # 폭력적 문서 아니면.
            p0Num += trainMatrix[i] # p1Num배열에서 해당 문서의 단어를 포함하는 인덱스 +1
            p0Denom += sum(trainMatrix[i]) # 해당 문서 내 단어 총갯수
    
    print "p0Num : %s" % p0Num
    print "p1Num : %s" % p1Num
    p1Vect = p1Num / p1Denom # 폭력적 단어 확률
    p0Vect = p0Num / p0Denom # 비폭력적 단어 확률
    
    return p0Vect, p1Vect, pAbusive


# 소스코드 출력
reload(bayes)
listOPosts, listClasses = bayes.loadDataSet()

myVocabList = bayes.createVocabList(listOPosts)

trainMat = []
for postinDoc in listOPosts:
    trainMat.append(bayes.setOfWords2Vec(myVocabList, postinDoc))
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

print "p0V : %s" % p0V
print "p1V : %s" % p1V
print "pAb : %s" % pAb

p0Num : [ 1.  1.  1.  0.  0.  1.  1.  1.  0.  1.  1.  1.  1.  0.  0.  2.  0.  0.
  1.  0.  1.  1.  0.  1.  1.  1.  0.  1.  0.  1.  1.  3.]
p1Num : [ 0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  1.  1.  1.
  0.  2.  0.  1.  1.  0.  2.  0.  3.  0.  1.  0.  0.  0.]
p0V : [ 0.04166667  0.04166667  0.04166667  0.          0.          0.04166667
  0.04166667  0.04166667  0.          0.04166667  0.04166667  0.04166667
  0.04166667  0.          0.          0.08333333  0.          0.
  0.04166667  0.          0.04166667  0.04166667  0.          0.04166667
  0.04166667  0.04166667  0.          0.04166667  0.          0.04166667
  0.04166667  0.125     ]
p1V : [ 0.          0.          0.          0.05263158  0.05263158  0.          0.
  0.          0.05263158  0.05263158  0.          0.          0.
  0.05263158  0.05263158  0.05263158  0.05263158  0.05263158  0.
  0.10526316  0.          0.05263158  0.05263158  0.          0.10526316
  0.          0.15789474  0.          0.0526315

### 4.5.3 검사: 실제 조건을 반영하기 위해 분류기 수정하기

#### 리스팅 4.3 나이브 베이스 분류 함수_p93

In [28]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts, listClasses = bayes.loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = bayes.trainNB0(array(trainMat), array(listClasses))

    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb)

    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry, 'Classified as: ', classifyNB(thisDoc, p0V, p1V, pAb)


# 소스코드 출력
bayes.testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


### 4.5.4 준비: 중복 단어 문서 모델

#### 리스팅 4.4 나이브 베이스 중복 단어 모델_p94

In [30]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

## 4.6 예제: 스팸 이메일 분류하기

### 4.6.1 준비: 텍스트 토큰 만들기

In [31]:
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

In [33]:
import re
regEx = re.compile('\\W*')
listOfTokens = regEx.split(mySent)
print listOfTokens

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'ever', 'laid', 'eyes', 'upon', '']


In [35]:
[tok for tok in listOfTokens if len(tok) > 0]

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [34]:
[tok.lower() for tok in listOfTokens if len(tok) > 0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [4]:
# emailText = open('email/ham/6.txt').read()

emailText = open('C:\Users\BMW\Ch04_NaiveBayes\email\ham\6.txt').read()
listOfTokens = regEx.split(emailText)

IOError: [Errno 22] invalid mode ('r') or filename: 'C:\\Users\\BMW\\Ch04_NaiveBayes\\email\\ham\x06.txt'

### 4.6.2 검사: 나이브 베이스로 교차 검증하기

#### 리스팅 4.5 나이브 베이스로 교차 검증하기_p97

In [16]:
def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 
    
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = range(50); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print "classification error",docList[docIndex]
    print 'the error rate is: ',float(errorCount)/len(testSet)
    #return vocabList,fullText

In [5]:
bayes.spamTest()

NameError: name 'bayes' is not defined

## 4.7 예제: 나이브 베이스를 사용하여 개인 광고에 포함된 지역 특색 도출하기

### 4.7.1 수집: RSS 피드 불러오기

In [7]:
import feedparser
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')

ImportError: No module named feedparser

In [8]:
ny['entries']
len(ny['entries'])

NameError: name 'ny' is not defined

#### 리스팅 4.6 RSS 피드 분류기와 자주 발생하는 단어 제거 함수_p101

In [18]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq = sorted(freqDict.iteritems(), key=operator.itemgetter(1), reverse=True) 
    return sortedFreq[:30]       

def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet=[]           #create test set
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print 'the error rate is: ',float(errorCount)/len(testSet)
    return vocabList,p0V,p1V

### 4.7.2 분석: 지역적으로 사용되는 단어 표현하기

#### 리스팅 4.7 서술된 대부분의 단어를 표시하는 함수_p104

In [17]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print "SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**"
    for item in sortedSF:
        print item[0]
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print "NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**"
    for item in sortedNY:
        print item[0]